In [1]:
# Most importantly, select the kernal at least 3.11

In [110]:
import os, subprocess
import torch, struct, os, psutil, subprocess, time, threading
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf
import json, ezkl
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset

# DNN Models

## Prepare

In [111]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Convert to PyTorch tensors
test_images_pt = torch.tensor(test_images).float()
test_labels_pt = torch.tensor(test_labels)
# Flatten and normalize the images
test_images_pt = test_images_pt.view(-1, 28*28) / 255.0  # Flatten and normalize

# Assuming test_images_pt is your PyTorch tensor with shape [num_samples, 784]
test_images_pt_reshaped = test_images_pt.view(-1, 1, 28, 28)  # Reshape to [num_samples, channels, height, width]

# Downsample images
test_images_pt_downsampled = F.interpolate(test_images_pt_reshaped, size=(14, 14), mode='bilinear', align_corners=False)

# Flatten the images back to [num_samples, 14*14]
test_images_pt_downsampled = test_images_pt_downsampled.view(-1, 14*14)

In [112]:
def evaluate_pytorch_model(model, datasets, labels):
    # Create TensorDataset for test data
    test_dataset = TensorDataset(datasets, labels)
    # Create a DataLoader for the test dataset
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [113]:
import subprocess, concurrent
import psutil
import time

def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    #print(f"Maximum memory used: {max_memory} MB")
    return max_memory

def execute_and_monitor(command, show = False):
    start_time = time.time()
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(monitor_memory, process.pid)
        stdout, stderr = process.communicate()
        max_memory = future.result()
    if show:
        print(f"Maximum memory used: {max_memory} MB")
        print("Total time:", time.time() - start_time)
    return stdout, stderr, max_memory


In [114]:
def benchmark_dnn(test_images, predictions, model, mode = "resources", output_folder='./tmp/'):
    data_path = os.path.join(output_folder, 'input.json')
    model_path = os.path.join(output_folder, 'network.onnx')

    sampled_data = test_images[0]
    
    with torch.no_grad():
        torch.onnx.export(model, 
                    sampled_data, 
                    model_path, 
                    export_params=True, 
                    opset_version=10, 
                    do_constant_folding=True, 
                    input_names=['input_0'], 
                    output_names=['output'])
    loss = 0
    mem_usage = []
    time_cost = []
    benchmark_start_time = time.time()

    for i, img in enumerate(test_images):
        print ("Process for image", i)
        start_time = time.time()
        # Convert the tensor to numpy array and reshape it for JSON serialization
        x = (img.cpu().detach().numpy().reshape([-1])).tolist()
        data = dict(input_data = [x])

        # Serialize data into file:
        json.dump(data, open(data_path, 'w'))

        command = ["python", "gen_proof.py", "--model", model_path, "--data", data_path, "--output", output_folder, "--mode", mode]
        #subprocess.run(command)
        stdout, _, usage = execute_and_monitor(command)
        pred = int(stdout[-2])

        if pred != predictions[i]:
            loss += 1
            print ("Loss happens on index", i, "predicted_class", pred)
        mem_usage.append(usage)
        time_cost.append(time.time() - start_time)

    print ("Total time:", time.time() - benchmark_start_time)
    return loss, mem_usage, time_cost

In [115]:
def benchmark_cnn(test_images, predictions, model, mode = "resources", output_folder='./tmp/'):
    data_path = os.path.join(output_folder, 'input.json')
    model_path = os.path.join(output_folder, 'network.onnx')

    loss = 0
    mem_usage = []
    time_cost = []
    benchmark_start_time = time.time()

    for i in range(len(test_images)):
        print ("Process for image", i)
        start_time = time.time()
        img = test_images[i:i+1]
        with torch.no_grad():
            torch.onnx.export(model, 
                        img, 
                        model_path, 
                        export_params=True, 
                        opset_version=10, 
                        do_constant_folding=True, 
                        input_names=['input_0'], 
                        output_names=['output'])
        
        # Convert the tensor to numpy array and reshape it for JSON serialization
        # x = (img.cpu().detach().numpy().reshape([-1])).tolist()
        # data = dict(input_data = [x])

        # # Serialize data into file:
        # json.dump(data, open(data_path, 'w'))

        command = ["python", "gen_proof.py", "--model", model_path, "--data", data_path, "--output", output_folder, "--mode", mode]
        
        stdout, _, usage = execute_and_monitor(command)
        pred = int(stdout[-2])

        if pred != predictions[i]:
            loss += 1
            print ("Loss happens on index", i, "predicted_class", pred)
        mem_usage.append(usage)
        time_cost.append(time.time() - start_time)

    print ("Total time:", time.time() - benchmark_start_time)
    return loss, mem_usage, time_cost

In [116]:
csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")


File '../../benchmarks/benchmark_results.csv' already exists.


## Benchmark for 196_25_10 DNN Model

In [117]:
# After login
arch_folder = "input-dense-dense/"
layers = [196,25,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/" + arch_folder + model_name

state_dict = torch.load(model_path + ".pth")

output_folder = './tmp/' + "_".join([str(x) for x in layers]) + "/"
os.makedirs(output_folder, exist_ok=True)

In [118]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2])  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt_downsampled)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [119]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt_downsampled, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 95.41000000%


### Benchmark (resource mode)

In [120]:
test_size = 10
loss_, mem_usage_, time_cost_ = benchmark_dnn(test_images_pt_downsampled[:test_size], predicted_labels[:test_size], model_pt)

Process for image 0
Process for image 1


Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Total time: 7.243288040161133


: 

In [77]:
new_row = {
    'Framework': ['ezkl (pytorch)'],
    'Architecture': ['Input-Dense-Dense (196x25x10'],
    '# Layers': [3],
    '# Parameters': [5185],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss_/test_size*100],
    'Avg Memory Usage (MB)': [sum(mem_usage_) / len(mem_usage_)],
    'Std Memory Usage': [pd.Series(mem_usage_).std()],
    'Avg Proving Time (s)': [sum(time_cost_) / len(time_cost_)],
    'Std Proving Time': [pd.Series(time_cost_).std()]
}

new_row_df = pd.DataFrame(new_row)

In [12]:
df = pd.read_csv(csv_path)

df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
2,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10,4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
4,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10,3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
5,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
6,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
7,zkml (tensorflow),Input-Dense-Dense (196x24x14x10,4,5228,2500,0.00,2338.480847,13.611456,22.168541,0.528263
8,ezkl (pytorch),Input-Dense-Dense (196x25x10,3,5185,10000,4.11,91.608069,0.756873,0.660479,0.025701


### Benchmark (accuracy mode)

In [99]:
test_size = 1000
loss_, mem_usage_, time_cost_ = benchmark_dnn(test_images_pt_downsampled[:test_size], predicted_labels[:test_size], model_pt, mode="accuracy")

Process for image 0


Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Process for image 10
Process for image 11
Process for image 12
Process for image 13
Process for image 14
Process for image 15
Process for image 16
Process for image 17
Process for image 18
Process for image 19
Process for image 20
Process for image 21
Process for image 22
Process for image 23
Process for image 24
Process for image 25
Process for image 26
Process for image 27
Process for image 28
Process for image 29
Process for image 30
Process for image 31
Process for image 32
Process for image 33
Process for image 34
Process for image 35
Process for image 36
Process for image 37
Process for image 38
Process for image 39
Process for image 40
Process for image 41
Process for image 42
Process for image 43
Process for image 44
Process for image 45
Process for image 46
Process for image 47
Process for image 48
P

In [100]:
new_row = {
    'Framework': ['ezkl-accuracy (pytorch)'],
    'Architecture': ['Input-Dense-Dense (196x25x10)'],
    '# Layers': [3],
    '# Parameters': [5185],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss_/test_size*100],
    'Avg Memory Usage (MB)': [sum(mem_usage_) / len(mem_usage_)],
    'Std Memory Usage': [pd.Series(mem_usage_).std()],
    'Avg Proving Time (s)': [sum(time_cost_) / len(time_cost_)],
    'Std Proving Time': [pd.Series(time_cost_).std()]
}

new_row_df = pd.DataFrame(new_row)

In [101]:
df = pd.read_csv(csv_path)

df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
2,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
4,circomlib-ml (tensorflow),Input-Dense-Dense-Dense (196x24x14x10),4,5228,2500,0.88,315.435559,1.690681,0.452942,0.031613
5,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
6,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
7,zkml (tensorflow),Input-Dense-Dense-Dense (196x24x14x10),4,5228,2500,0.00,2338.480847,13.611456,22.168541,0.528263
8,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
9,zkml (tensorflow),Input-Conv2d-Conv2d-Dense (784x6x16x10),4,5142,1000,0.10,1764.619016,12.589448,15.618183,0.825017


## Benchmark for 196_24_14_10 DNN Model

In [79]:
# After login
arch_folder = "input-dense-dense-dense/"
layers = [196,24,14,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/" + arch_folder + model_name

state_dict = torch.load(model_path + ".pth")

output_folder = './tmp/' + "_".join([str(x) for x in layers]) + "/"
os.makedirs(output_folder, exist_ok=True)

In [80]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2]) 
        self.fc3 = nn.Linear(layers[2], layers[3]) 

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt_downsampled)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [81]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt_downsampled, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 95.56000000%


### Benchmark - resource

In [88]:
test_size = 10
loss_, mem_usage_, time_cost_ = benchmark_dnn(test_images_pt_downsampled[:test_size], predicted_labels[:test_size], model_pt)

Process for image 0


Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Total time: 7.382734298706055


In [17]:
new_row = {
    'Framework': ['ezkl-accuracy (pytorch)'],
    'Architecture': ['Input-Dense-Dense (196x24x14x10'],
    '# Layers': [4],
    '# Parameters': [5228],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss_/test_size*100],
    'Avg Memory Usage (MB)': [sum(mem_usage_) / len(mem_usage_)],
    'Std Memory Usage': [pd.Series(mem_usage_).std()],
    'Avg Proving Time (s)': [sum(time_cost_) / len(time_cost_)],
    'Std Proving Time': [pd.Series(time_cost_).std()]
}

new_row_df = pd.DataFrame(new_row)

In [18]:
df = pd.read_csv(csv_path)

df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
2,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10,4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
4,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10,3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
5,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
6,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
7,zkml (tensorflow),Input-Dense-Dense (196x24x14x10,4,5228,2500,0.00,2338.480847,13.611456,22.168541,0.528263
8,ezkl (pytorch),Input-Dense-Dense (196x25x10,3,5185,10000,4.11,91.608069,0.756873,0.660479,0.025701
9,ezkl (pytorch),Input-Dense-Dense (196x24x14x10,4,5228,10000,15.71,91.589901,0.742728,0.693572,0.024197


## Benchmark for 784_56_10 DNN Model

In [19]:
# After login
arch_folder = "input-dense-dense/"
layers = [784,56,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/" + arch_folder + model_name

state_dict = torch.load(model_path + ".pth")

output_folder = './tmp/' + "_".join([str(x) for x in layers]) + "/"
os.makedirs(output_folder, exist_ok=True)

In [20]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2])  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [21]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 97.40000000%


### Benchmark

In [22]:
test_size = 10000
loss_, mem_usage_, time_cost_ = benchmark(test_images_pt[:test_size], predicted_labels[:test_size], model_pt)

Process for image 0
Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6


Process for image 7
Process for image 8
Process for image 9
Process for image 10
Process for image 11
Process for image 12
Process for image 13
Process for image 14
Process for image 15
Process for image 16
Process for image 17
Process for image 18
Loss happens on index 18 predicted_class 2
Process for image 19
Process for image 20
Process for image 21
Process for image 22
Process for image 23
Process for image 24
Process for image 25
Process for image 26
Process for image 27
Process for image 28
Process for image 29
Process for image 30
Process for image 31
Process for image 32
Process for image 33
Loss happens on index 33 predicted_class 0
Process for image 34
Process for image 35
Process for image 36
Process for image 37
Process for image 38
Process for image 39
Process for image 40
Process for image 41
Process for image 42
Process for image 43
Process for image 44
Process for image 45
Process for image 46
Process for image 47
Process for image 48
Loss happens on index 48 predicted_

In [25]:
new_row = {
    'Framework': ['ezkl (pytorch)'],
    'Architecture': ['Input-Dense-Dense (784x56x10)'],
    '# Layers': [3],
    '# Parameters': [44543],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss_/test_size*100],
    'Avg Memory Usage (MB)': [sum(mem_usage_) / len(mem_usage_)],
    'Std Memory Usage': [pd.Series(mem_usage_).std()],
    'Avg Proving Time (s)': [sum(time_cost_) / len(time_cost_)],
    'Std Proving Time': [pd.Series(time_cost_).std()]
}

new_row_df = pd.DataFrame(new_row)

In [ ]:
df = pd.read_csv(csv_path)

df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

In [27]:
df = pd.read_csv(csv_path)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
2,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
4,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10,3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
5,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
6,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
7,zkml (tensorflow),Input-Dense-Dense (196x24x14x10),4,5228,2500,0.00,2338.480847,13.611456,22.168541,0.528263
8,ezkl (pytorch),Input-Dense-Dense (196x25x10),3,5185,10000,4.11,91.608069,0.756873,0.660479,0.025701
9,ezkl (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,10000,15.71,91.589901,0.742728,0.693572,0.024197


# CNN Models

In [102]:
import numpy as np
import torch

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images_tf = train_images / 255.0
test_images_tf = test_images / 255.0
train_images_tf = train_images_tf.reshape(train_images.shape[0], 28, 28, 1)
test_images_tf = test_images_tf.reshape(test_images.shape[0], 28, 28, 1)

train_images_tf_14 = tf.image.resize(train_images_tf, [14, 14]).numpy()
test_images_tf_14 = tf.image.resize(test_images_tf, [14, 14]).numpy()

In [103]:
# Convert to PyTorch format [batch_size, channels, height, width]
train_images_pt = torch.tensor(train_images_tf).permute(0, 3, 1, 2).float()
test_images_pt = torch.tensor(test_images_tf).permute(0, 3, 1, 2).float()


train_images_pt_14 =  torch.tensor(test_images_tf_14).permute(0, 3, 1, 2).float()
test_images_pt_14 =  torch.tensor(test_images_tf_14).permute(0, 3, 1, 2).float()

## Benchmark for 14_5_11_80_10_3 CNN Model

In [104]:
# After login
arch_folder = "input-conv2d-conv2d-dense-dense/"
layers = [14,5,11,80,10,3]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/" + arch_folder + model_name

state_dict = torch.load(model_path + ".pth")

output_folder = './tmp/' + "_".join([str(x) for x in layers]) + "/"
os.makedirs(output_folder, exist_ok=True)

In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F

layers = [14, 5, 11, 80, 10, 3]
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, layers[1], layers[-1]) 
        self.conv2 = nn.Conv2d(layers[1], layers[2], layers[-1]) 

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(11 * 2 * 2, layers[3]) # 256 * 120
        self.fc2 = nn.Linear(layers[3], layers[4])

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.relu(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.relu(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # TODO: figure out the resize, currently work on batch_size = 1
        batch_size = x.size(0)
        x = x.reshape(x.size(0),layers[2],-1)  # 16 output channels
        x = np.transpose(x, (0,2,1)).reshape(batch_size,-1)
        #x = x.reshape(batch_size,-1)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation function here, will use CrossEntropyLoss later
        return x

model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt_14)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [106]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt_14, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 97.07000000%


### Benchmark

In [107]:
test_size = 10000
loss_, mem_usage_, time_cost_ = benchmark_cnn(test_images_pt_14[:test_size], 
                                          predicted_labels[:test_size], model_pt)

Process for image 0
Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Process for image 10
Process for image 11
Process for image 12
Process for image 13
Process for image 14
Process for image 15
Process for image 16
Process for image 17
Process for image 18
Process for image 19
Process for image 20
Process for image 21
Process for image 22
Process for image 23
Process for image 24
Process for image 25
Process for image 26
Process for image 27
Process for image 28
Process for image 29
Process for image 30
Process for image 31
Process for image 32
Process for image 33
Process for image 34
Process for image 35
Process for image 36
Process for image 37
Process for image 38
Process for image 39
Process for image 40
Process for image 41
Process for image 42
Process for image 43
Process for image 44
Process for image 45
Process for image 46
Process for image 47
Pr

Process for image 78
Process for image 79
Process for image 80
Process for image 81
Process for image 82
Process for image 83
Process for image 84
Process for image 85
Process for image 86
Process for image 87
Process for image 88
Process for image 89
Process for image 90
Process for image 91
Process for image 92
Process for image 93
Process for image 94
Process for image 95
Process for image 96
Process for image 97
Process for image 98
Process for image 99
Process for image 100
Process for image 101
Process for image 102
Process for image 103
Process for image 104
Process for image 105
Process for image 106
Process for image 107
Process for image 108
Process for image 109
Process for image 110
Process for image 111
Process for image 112
Process for image 113
Process for image 114
Process for image 115
Process for image 116
Process for image 117
Process for image 118
Process for image 119
Process for image 120
Process for image 121
Process for image 122
Process for image 123
Process fo

In [108]:
new_row = {
    'Framework': ['ezkl-resource (pytorch)'],
    'Architecture': ['Input-Conv2d-Conv2d-Dense-Dense (196x5x11x80x10'],
    '# Layers': [5],
    '# Parameters': [4966],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss_/test_size*100],
    'Avg Memory Usage (MB)': [sum(mem_usage_) / len(mem_usage_)],
    'Std Memory Usage': [pd.Series(mem_usage_).std()],
    'Avg Proving Time (s)': [sum(time_cost_) / len(time_cost_)],
    'Std Proving Time': [pd.Series(time_cost_).std()]
}

new_row_df = pd.DataFrame(new_row)

In [109]:
df = pd.read_csv(csv_path)

df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
2,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
4,circomlib-ml (tensorflow),Input-Dense-Dense-Dense (196x24x14x10),4,5228,2500,0.88,315.435559,1.690681,0.452942,0.031613
5,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
6,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
7,zkml (tensorflow),Input-Dense-Dense-Dense (196x24x14x10),4,5228,2500,0.00,2338.480847,13.611456,22.168541,0.528263
8,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
9,zkml (tensorflow),Input-Conv2d-Conv2d-Dense (784x6x16x10),4,5142,1000,0.10,1764.619016,12.589448,15.618183,0.825017
